In [1]:
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import pickle
from glob import glob

In [2]:
path = '/home/ubuntu/intermediate-result-new-select-editors/warring-camps-threshold-100/revert-index-info/'

In [3]:
all_folders = glob(path + "*")

In [4]:
df = pd.read_parquet(all_folders[0])

In [5]:
for f in all_folders[1:]:
    curr_df = pd.read_parquet(f)
    df = pd.concat([df,curr_df])

In [6]:
# with open('../../all-data-article-clustering/articles-100-threshold.pkl' ,'rb') as f:
#     selected_articles = pickle.load(f) 

In [7]:
# df = df[df['page.title'].isin(selected_articles)]

In [8]:
user_group = pd.read_csv('result/all-data-editor-cluster-result.csv', sep="\t",encoding='utf-16').set_index('contributor.username')

In [9]:
revert_info = df

In [10]:
revert_info = revert_info.merge(user_group,left_on='user1',right_index=True).rename(columns={'group':'group1'})

In [11]:
revert_info = revert_info.merge(user_group,left_on='user2',right_index=True).rename(columns={'group':'group2'})

In [12]:
mutual_revert = pd.read_csv('result/mutual-and-min-revert.csv', sep="\t",encoding='utf-16')

In [13]:
mutual_revert_info = revert_info[['page.title','group1','group2']]

In [14]:
# switch group1 and group2 if group1 is greater than group2
def switch_group(d):
    g1 = d['group1']
    g2 = d['group2']
    if g1 > g2:
        tmp = g1
        d['group1'] = g2
        d['group2'] = tmp
    return d

In [15]:
mutual_revert_info = mutual_revert_info.apply(switch_group,axis=1)

In [16]:
res = mutual_revert.merge(mutual_revert_info,on=['group1','group2'])

In [17]:
fin = res.groupby(['group1','group2','page.title']).agg({'page.title':'count'}).rename(columns={'page.title':'mutual_revert'})

In [18]:
fin.reset_index(inplace=True)

In [19]:
fin.to_csv('result/group-pair-article-mutual-revert-info.csv', index=False,sep="\t",encoding='utf-16')